In [1]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from featfuncs import add_global_indices_feed

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from featfuncs.ipynb
importing Jupyter notebook from india_calendar.ipynb


In [3]:
from rulestrats import AdaMomCMOADF, GapBet

importing Jupyter notebook from rulestrats.ipynb


In [4]:
def do_nothing(dfD):
    empty={t:0 for t in dfD}
    return empty,empty,empty
def always_buy(dfD):
    buy={t:1 for t in dfD}
    empty={t:0 for t in dfD}
    return buy,empty,empty
def always_sell(dfD):
    sell={t:-1 for t in dfD}
    empty={t:0 for t in dfD}
    return sell,empty,empty

Host Service via Anvil

In [5]:
import anvil.server

In [6]:
# DeepTrader:
anvil.server.connect("STKYCIXUMGFV2CURGMCMOFUV-MBK5ZXFC4MCKYKOE")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [5]:
momStrat=MomStrat()

NameError: name 'MomStrat' is not defined

In [6]:
momStrat=AdaMomCMOADF()

In [ ]:
@anvil.server.callable
def check_entry_batch_mom2(dfD):
    global momStrat
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Check(momStrat,dfDloc)

In [ ]:
@anvil.server.callable
def check_exit_batch_mom2(dfD,posf):
    global momStrat
    posloc=pd.DataFrame(posf)
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Exit(momStrat,dfDloc,posloc)

Strategy Development: ADF Momentum

In [6]:
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate

importing Jupyter notebook from backtest.ipynb
importing Jupyter notebook from feed_env.ipynb
importing Jupyter notebook from validation.ipynb


In [7]:
data=pd.read_csv('./capvol100.csv')

In [8]:
tickers=list(data.iloc[0:10]['ticker'].values)

In [9]:
feed=BackFeed(tickers=tickers,nd=3,nw=1)

In [10]:
add_addl_features_feed(feed,tickers=feed.tickers)

In [11]:
add_sym_feature_feed(feed,tickers=feed.tickers)

In [12]:
add_global_indices_feed(feed)

In [13]:
feed.gdata

{'21-Oct-2022': [{'Open_^NSEI': 1.0,
   'High_^NSEI': 1.0092435206628414,
   'Low_^NSEI': 0.9998794927755639,
   'Close_^NSEI': 1.0080840730141503,
   'Volume_^NSEI': 0.8740952623756619,
   'Open_^NYA': 0.989925586426494,
   'High_^NYA': 1.0004606531446092,
   'Low_^NYA': 0.9801343851233572,
   'Close_^NYA': 0.9823460192699404,
   'Volume_^NYA': 1.1989233047831307,
   'Open_LSEG.L': 0.9954496788008566,
   'High_LSEG.L': 1.0001054718867104,
   'Low_LSEG.L': 0.9692184154175589,
   'Close_LSEG.L': 0.9925053533190579,
   'Volume_LSEG.L': 0.6986482844205889,
   'Open_^IXIC': 0.9970157647832472,
   'High_^IXIC': 1.0138446953380427,
   'Low_^IXIC': 0.9892640619237258,
   'Close_^IXIC': 0.9930332519539592,
   'Volume_^IXIC': 0.9315169680571723}],
 '25-Oct-2022': [{'Open_^NSEI': 1.0,
   'High_^NSEI': 1.0023229792365629,
   'Low_^NSEI': 0.9983678028800979,
   'Close_^NSEI': 0.9996842862540305,
   'Volume_^NSEI': 0.15758929009176598,
   'Open_^NYA': 1.0,
   'High_^NYA': 1.0088581446468732,
   'Lo

In [27]:
def global_direction(gdata):
    global_tickers=['^NYA','LSEG.L','^IXIC']
    direction={}
    for g in global_tickers:
        direction[g]=gdata['Close_'+g]-gdata['Open_'+g]
    return 100*sum([direction[k] for k in direction])/len(direction)
def domestic_direction(gdata):
    tickers=['^NSEI']
    direction={}
    for g in tickers:
        direction[g]=gdata['Close_'+g]-gdata['Open_'+g]
    return 100*sum([direction[k] for k in direction])/len(direction)

In [18]:
feed.gdata['21-Oct-2022']

[{'Open_^NSEI': 1.0,
  'High_^NSEI': 1.0092435206628414,
  'Low_^NSEI': 0.9998794927755639,
  'Close_^NSEI': 1.0080840730141503,
  'Volume_^NSEI': 0.8740952623756619,
  'Open_^NYA': 0.989925586426494,
  'High_^NYA': 1.0004606531446092,
  'Low_^NYA': 0.9801343851233572,
  'Close_^NYA': 0.9823460192699404,
  'Volume_^NYA': 1.1989233047831307,
  'Open_LSEG.L': 0.9954496788008566,
  'High_LSEG.L': 1.0001054718867104,
  'Low_LSEG.L': 0.9692184154175589,
  'Close_LSEG.L': 0.9925053533190579,
  'Volume_LSEG.L': 0.6986482844205889,
  'Open_^IXIC': 0.9970157647832472,
  'High_^IXIC': 1.0138446953380427,
  'Low_^IXIC': 0.9892640619237258,
  'Close_^IXIC': 0.9930332519539592,
  'Volume_^IXIC': 0.9315169680571723}]

In [28]:
global_direction(feed.gdata['21-Oct-2022'][0])

-0.483546848921342

In [29]:
domestic_direction(feed.gdata['21-Oct-2022'][0])

0.8084073014150261

In [14]:
import pickle

In [15]:
# with open('../../temp_data/btfeed.pickle','wb') as f: pickle.dump(feed,f)

In [11]:
with open('../../temp_data/btfeed.pickle','rb') as f: feed=pickle.load(f)

In [23]:
gapbet=GapBet(direction=-1)

In [17]:
adamom=AdaMomCMOADF()

In [18]:
import importlib
import sys,os

In [14]:
importlib.reload(sys.modules['feed_env'])
from feed_env import Episode

importing Jupyter notebook from feed_env.ipynb


In [14]:
importlib.reload(sys.modules['backtest'])
from backtest import Backtest

importing Jupyter notebook from backtest.ipynb


In [22]:
importlib.reload(sys.modules['rulestrats'])
from rulestrats import AdaMomCMOADF,GapBet

importing Jupyter notebook from rulestrats.ipynb


In [16]:
# def save_func(df): return(df.iloc[-1]['CMO_14'])

In [24]:
bt=Backtest(feed,tickers=feed.tickers,add_features=False,target=.5,stop=.5,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=True,save_dfs=True,
            save_func=None)

In [27]:
bt.run_all(tickers=feed.tickers,model=adamom,verbose=False)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
bt.returns

{'NESTLEIND.NS': 0.8173776696842956,
 'ULTRACEMCO.NS': -0.5695858653864828,
 'NTPC.NS': -1.7962502197683683,
 'DMART.NS': -0.014734645213544652,
 'ADANITRANS.NS': -0.33599666635171693,
 'KOTAKBANK.NS': 2.128687069220879}

In [21]:
bt.returns

{'NESTLEIND.NS': -2.8570876409569874,
 'ULTRACEMCO.NS': -0.08909594517244862,
 'NTPC.NS': 0.0,
 'DMART.NS': 0.0,
 'ADANITRANS.NS': 0.0,
 'KOTAKBANK.NS': 0.0}

In [ ]:
# for i in range(10): 
#     dfD={'TCS.NS':feed.getDataN('TCS.NS',d='14-Oct-2022')}
#     # print('entry',adamom.check_entry_batch(dfD))
#     # adamom.regime='tr'
#     # print('exit long tr',adamom.exit_predicate({'quant':1},dfD['TCS.NS']))
#     # print('exit short tr',adamom.exit_predicate({'quant':-1},dfD['TCS.NS']))
#     adamom.regime='mr'
#     print(dfD['TCS.NS'].iloc[-1]['CMO_14'])
#     print('exit long mr',adamom.exit_predicate({'quant':1},dfD['TCS.NS']))
#     print('exit short mr',adamom.exit_predicate({'quant':-1},dfD['TCS.NS']))
#     feed.step()

In [ ]:
feed.ndata